# Data Cleaning

In [363]:
import pandas as pd
import numpy as np
#import seaborn as sns
import json
import pprint
import datetime
import time
from nltk.corpus import stopwords
import string
from collections import defaultdict
#from datetime import timedelta  

In [364]:
with open('data.json') as f:
    data = json.load(f)

In [365]:
pprint.pprint(data['hdfc'][0:1])

[{'content': ['The second largest private lender\xa0',
              ' on Monday launched co-branded medical benefits card with ',
              ', which allows corporates to disburse medical allowances to '
              'their employees and has insurance cover for accidents.',
              'With the HDFC Bank-Apollo Medical Benefits Cards, corporates '
              'can load the specified allowance onto the card each month, '
              'which can be used by employees for medical expenditure '
              'Visa/MasterCard outlets. The bank claimed that this is the '
              'first-of-its kind product in the country.',
              'The card comes with a free accidental death insurance of Rs 3 '
              'lakh and accident hospitalisation insurance of Rs 30,000 apart '
              'from discounts at Apollo network including Apollo Hospitals, '
              'Apollo Pharmacies, Apollo Clinics and Apollo White Dental.',
              '"The HDFC Bank-Apollo Medical B

In [366]:
# date parser
count=0
for i in data['hdfc'][:]:
    #pprint.pprint(i['date'].strip())
    try:
        i['date']=datetime.datetime.strptime(i['date'].strip(), '%B %d, %Y').date()
        count+=1  
        #print(i['date'])
        #print(type(i['date']))
    except ValueError as e:
        print('ValueError:', e,'with date',i['date'])
        # pprint.pprint(i['date'][0:5])
print(count)

467


In [367]:
for i in data['hdfc'][:1]:
    print(type(i['date']))

<class 'datetime.date'>


In [368]:
# time parser
count = 0
for i in data['hdfc'][:]:
    
    try:
        i['time'] = i['time'].strip().split()[1]+' '+i['time'].strip().split()[2] # changes time in the format 7;30 pm 
        i['time'] = datetime.datetime.strptime(i['time'],'%I:%M %p').time()
        
        if i['time'] > datetime.datetime.strptime('15:30','%H:%M').time():
            #print('before date',i['date'])
            i['date']=i['date']+datetime.timedelta(days=1)
            #print('after date',i['date'])
        count+=1
       
    except (ValueError,IndexError) as e:
        print('IndexError:', e,'with time',i['time'])
print(count)
  

IndexError: list index out of range with time                                                                   
466


In [369]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
   

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
   # print(nopunc)

    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [370]:
# content parser
# cleans the content
for i in data['hdfc'][0:]:
    try:
        i['content']=text_process(i['content'])
        #print(text)
            
    except:
        print('\n*3 The error causing content',i['content'])

In [394]:
## check to see how the data looks!
for i in data['hdfc'][0:2]:
    print('\n',i['time'])
    print('\n',i['date'])
    print('\n',i['content'])
    


 19:12:00

 2015-05-19

 ['second', 'largest', 'private', 'lender', 'Monday', 'launched', 'co-branded', 'medical', 'benefits', 'card', ',', 'allows', 'corporates', 'disburse', 'medical', 'allowances', 'employees', 'insurance', 'cover', 'accidents.With', 'HDFC', 'Bank-Apollo', 'Medical', 'Benefits', 'Cards,', 'corporates', 'load', 'specified', 'allowance', 'onto', 'card', 'month,', 'used', 'employees', 'medical', 'expenditure', 'Visa/MasterCard', 'outlets.', 'bank', 'claimed', 'first-of-its', 'kind', 'product', 'country.The', 'card', 'comes', 'free', 'accidental', 'death', 'insurance', 'Rs', '3', 'lakh', 'accident', 'hospitalisation', 'insurance', 'Rs', '30,000', 'apart', 'discounts', 'Apollo', 'network', 'including', 'Apollo', 'Hospitals,', 'Apollo', 'Pharmacies,', 'Apollo', 'Clinics', 'Apollo', 'White', 'Dental."The', 'HDFC', 'Bank-Apollo', 'Medical', 'Benefits', 'Card', 'offers', 'easy', 'convenient', 'way', 'corporates', 'save', 'costs', 'eliminate', 'administrative', 'issues', 'di

In [372]:
# converting data to dictionary with key as date and value as news
news_dictionary=defaultdict(list)

for i in data['hdfc'][:]:
    news_dictionary[i['date']].extend(i['content'])

print(len(news_dictionary.keys()))

In [376]:
# reference cell for empty dates
"""
print(news_dictionary[datetime.date(2015, 11, 13)])

empty_keys=[datetime.date(2015, 11, 13),
 datetime.date(2015, 9, 25),
 datetime.date(2015, 8, 3),
 datetime.date(2017, 3, 17),
 datetime.date(2017, 3, 31),
 datetime.date(2017, 3, 30),
 datetime.date(2017, 7, 24),
 datetime.date(2017, 5, 31),
 datetime.date(2017, 6, 20),
 datetime.date(2017, 6, 23)]

for i in empty_keys:
    news_dictionary.pop(i)
  
#len(news_dictionary[datetime.date(2017, 5, 31)])
"""

[]


'\nempty_keys=[datetime.date(2015, 11, 13),\n datetime.date(2015, 9, 25),\n datetime.date(2015, 8, 3),\n datetime.date(2017, 3, 17),\n datetime.date(2017, 3, 31),\n datetime.date(2017, 3, 30),\n datetime.date(2017, 7, 24),\n datetime.date(2017, 5, 31),\n datetime.date(2017, 6, 20),\n datetime.date(2017, 6, 23)]\n\nfor i in empty_keys:\n    news_dictionary.pop(i)\n  \n#len(news_dictionary[datetime.date(2017, 5, 31)])\n'

In [378]:
# getting input values for the yahoo api
k=news_dictionary.keys()
max_date=max(k)
min_date=min(k)

In [379]:
# Financial data:
import pandas_datareader.data as web   # Package and modules for importing data; this code may change depending on pandas version
import datetime
 
# We will look at stock prices over these dates
start = min_date
end =  max_date
bank_name = 'HDFCBANK.NS' 

bank_fin = web.DataReader("HDFCBANK.NS", "yahoo", start, end)
 
type(bank_fin)

pandas.core.frame.DataFrame

In [411]:
#Bank data as df
bank_fin.head()
bank_fin['P/L']=bank_fin['Close']-bank_fin['Open']
bank_fin.reset_index(inplace=True)
print(len(bank_fin['P/L'].values))# number of P/L values

1413


In [383]:
# initializing for the next cell
news_date_list = list(news_dictionary.keys())
bank_date_list = list(bank_fin['Date'])

pandas._libs.tslibs.timestamps.Timestamp

In [384]:
# Picking out P/L for those dates with news articles
# This accounts for news that has in the weekends too!
# News on sat and sun are corelated to P/L on Monday (Works for any other holidays with two days gap too)

P_L_dict = {}
test_list = []
try:
    for i in range(len(bank_fin['Date'])):
        if bank_fin['Date'][i].date() in news_date_list:
            #print(date_list[i])
            P_L_dict[bank_fin['Date'][i].date()] = bank_fin['P/L'][i]  

    for holiday_dates in news_date_list:
        if holiday_dates not in bank_fin.values:
            for i in range(len(bank_fin['P/L'])):
                if holiday_dates + datetime.timedelta(days=1) == bank_fin['Date'][i]:
                    P_L_dict[holiday_dates] = bank_fin['P/L'][i]
                    break
                elif holiday_dates + datetime.timedelta(days=2) == bank_fin['Date'][i]:
                    P_L_dict[holiday_dates] = bank_fin['P/L'][i]
                    break
        
except (IndexError,ValueError) as e:
    print('This is the error',e)
    

for i in sorted(P_L_dict)[:5]:
    print(i,':',P_L_dict[i])

print(len(P_L_dict))

2015-05-15 : -2.4000244140625
2015-05-19 : 0.600006103515625
2015-05-26 : 1.47503662109375
2015-06-02 : -14.800018310546875
2015-06-11 : -8.699981689453125


324

In [385]:
# function to remove persistant punctuations
def text_remove_punctuations(news):
    """
    The function text_remove_punctuation:
    1. Removes remaining commas
    """
    cleaned_news = [word for word in news if word not in string.punctuation]
    return cleaned_news

In [386]:
# news cleaned with the extra commas! Yay!
for v in news_dictionary:
    news_dictionary[v] = text_remove_punctuations(news_dictionary[v])
    

In [387]:
# labeling the positive and negative news and creating DF:
df = pd.DataFrame()

for i in news_dictionary:
    try:
        if P_L_dict[i]>=0:
            df = df.append({'news': news_dictionary[i], 'label': int(1)}, ignore_index=True)
                

        elif P_L_dict[i]<0:
            df = df.append({'news': news_dictionary[i], 'label': int(0)}, ignore_index=True)
                
    except:
        print('This is date that is causing trouble',i )

print(df.head())
print(len(df))

This is date that is causing trouble 2019-04-19
This is date that is causing trouble 2020-05-01
This is date that is causing trouble 2018-03-29
This is date that is causing trouble 2015-09-25
This is date that is causing trouble 2017-04-29
   label                                               news
0    1.0  [second, largest, private, lender, Monday, lau...
1    1.0  [State-run, Tuesday, cut, base, rate, minimum,...
2    0.0  [First, blocks,, state-owned, Tuesday, cut, ba...
3    0.0  [Thursday, joined, private, sector, rivals, la...
4    0.0  [HDFCB, reported, net, profit, INR, 46.0bn, 1Q...


324

In [389]:
# checking the consistency of length of first news after all this processing
print(len(df['news'][0]))
for i in news_dictionary:
    print(len(news_dictionary[i]))
    break

96
96


In [ ]:
# checking for the consistency in number of news after all these processing
print(len(news_dictionary))
print(len(news_date_list))

# Data modeling

In [393]:
# removing empty news finally!
df['news'].replace('', np.nan, inplace=True)
df['news'].dropna()
df.head()

,label,news
0,1.0,"[second, largest, private, lender, Monday, lau..."
1,1.0,"[State-run, Tuesday, cut, base, rate, minimum,..."
2,0.0,"[First, blocks,, state-owned, Tuesday, cut, ba..."
3,0.0,"[Thursday, joined, private, sector, rivals, la..."
4,0.0,"[HDFCB, reported, net, profit, INR, 46.0bn, 1Q..."


In [402]:
#creating a identity function to pass into countvectorizer
def identity(news):
    return news

from sklearn.feature_extraction.text import CountVectorizer
# Vectorization of words, Might take awhile...
bow_transformer = CountVectorizer(analyzer=identity).fit(df['news'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

#Tokenization or vectorization of all the generated news
news_bow = bow_transformer.transform(df['news'])

print('Shape of Sparse Matrix: ', news_bow.shape)
print('Amount of Non-Zero occurences: ', news_bow.nnz)

#calculating sparsity
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

17847
Shape of Sparse Matrix:  (324, 17847)
Amount of Non-Zero occurences:  57988
sparsity: 1


In [406]:
# creating train and test data:
from sklearn.model_selection import train_test_split

news_train, news_test, label_train, label_test = \
train_test_split(df['news'], df['label'], test_size=0.3)

print(len(news_train), len(news_test), len(news_train) + len(news_test))

226 98 324


In [407]:
# making a pipeline of the above procedure
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=identity)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [408]:
pipeline.fit(news_train,label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function identity at 0x7fb081569510>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [409]:
#checking the predictions
predictions = pipeline.predict(news_test)
print(predictions[0:10])
print(label_test[0:10])

[1. 0. 1. 1. 1. 1. 1. 0. 1. 1.]
174    1.0
242    1.0
248    0.0
3      0.0
245    1.0
108    0.0
48     1.0
79     1.0
186    1.0
302    0.0
Name: label, dtype: float64


In [410]:
# checking the classification report
from sklearn.metrics import classification_report
print(classification_report(predictions,label_test))

              precision    recall  f1-score   support

         0.0       0.33      0.43      0.38        37
         1.0       0.58      0.48      0.52        61

    accuracy                           0.46        98
   macro avg       0.46      0.45      0.45        98
weighted avg       0.49      0.46      0.47        98

